## Checks Availability of the KanjiDB

In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  DatabaseHealth is running.                  /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _SLACK_SYSTEMCHANNEL_NAME = config['analytics']['systemchannelname']
  _SLACK_SYSTEMCHANNEL_DBID = int(config['analytics']['systemchannelid'])

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  DatabaseHealth is running.                  /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [2]:
def postMessageToSlack(blockmessage):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = _SLACK_SYSTEMCHANNEL_NAME    
    response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      logger(_LOG_ERROR, "Error posting message to Slack channel")
      logger(_LOG_ERROR, blockmessage)
      logger(_LOG_ERROR, response)
    else:
      logger(_LOG_INFO, "Ok posting message to Slack channel")

In [3]:
#Ticket Type
DATABASE_API = 10007
DATABASE_CONNECTION = 10008
DATABASE_NODE = 20012

_PREFERRED_IMAGE = "https://www.dropbox.com/s/ntr225t1xwyzsi7/db-missing.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE  

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
  query = "SELECT node.name, location.idlocation, location. description \
                 FROM kanji_node node JOIN kanji_location location ON node.location_id=location.idlocation \
                 WHERE idnode={} LIMIT 1".format(DATABASE_NODE)

  df = pd.read_sql(query, conn)
  nodename = df['name'][0]
  locationid = df['idlocation'][0]
  location = df['description'][0]
  mentions = ""

  if len(df.index) == 1:
    logger(_LOG_INFO, "PostgreSQL is Ok")
  else:
    logger(_LOG_INFO, "PostgreSQL is not Ok")
    description = " response = {}".format(len(df.index))    
      
    #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
    openTicket = kt.ticketExists(conn, DATABASE_NODE, DATABASE_API, [kt._OPEN_STATUS, kt._WORKING_STATUS])
    if openTicket is None:
      ticketid = kt.openticket(conn, DATABASE_NODE, locationid, description, 2, 3, DATABASE_API, _SLACK_SYSTEMCHANNEL_DBID)
      ts = kt.slackticket(nodename, location, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_SYSTEMCHANNEL_NAME, ticketid, 0)
      kt.updateTicket(conn, ticketid, ts)  
      logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
    else:
      logger(_LOG_INFO, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))
else:
  print("You are not connected to the database.")
  blockmessage = json.loads(messagetemplate)
  blockmessage[0]["accessory"]["image_url"] = locationimageurl
  blockmessage[0]["text"]["text"] = "{}".format("Database connection error.\n(non-ticketable)")
  postMessageToSlack(blockmessage)

logger(_LOG_INFO, "DatabaseHealth Done!")  

Welcome to Jupyter Notebook.  You are connected to the Kanji database!
PostgreSQL is Ok
DatabaseHealth Done!
